<a href="https://colab.research.google.com/github/anelsamadulla/energy_consumption/blob/main/%D0%9E%D1%82%D0%BA%D1%80%D1%8B%D1%82%D0%B8%D0%B5_%D0%B4%D0%B2%D0%B5%D1%80%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
THRESHOLD_MIN = 3  # порог тревоги/убытка

files = {
    "Дверь: Морозильник 1": "sample_data/Морозильник_1.csv",
    "Дверь: Холодильник 1": "sample_data/Холодильник_1.csv",
    "Дверь: Холодильник 2": "sample_data/Холодильник_2.csv",
}

def door_loss_events(csv_path, threshold_min=3):
    df = pd.read_csv(csv_path, sep=";")
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    df = df.sort_values("Timestamp").reset_index(drop=True)

    # open -> bool (на всякий)
    if df["open"].dtype != bool:
        df["open"] = df["open"].astype(str).str.lower().isin(["true", "1", "open", "opened", "да"])

    intervals = []
    open_start = None

    for ts, is_open in zip(df["Timestamp"], df["open"]):
        if is_open and open_start is None:
            open_start = ts
        elif (not is_open) and open_start is not None:
            intervals.append((open_start, ts))
            open_start = None

    # если файл закончился на открытой двери
    if open_start is not None:
        intervals.append((open_start, df["Timestamp"].iloc[-1]))

    events = []
    for start, end in intervals:
        dur_min = (end - start).total_seconds() / 60
        is_loss = dur_min >= threshold_min
        events.append({
            "start": start,
            "end": end,
            "duration_min": dur_min,
            "is_loss": is_loss,
            "over_threshold_min": max(0, dur_min - threshold_min),  # “сверх 3 минут”
        })

    return pd.DataFrame(events)

# считаем по всем дверям
all_events = []
for door, path in files.items():
    ev = door_loss_events(path, threshold_min=THRESHOLD_MIN)
    ev["door"] = door
    all_events.append(ev)

events = pd.concat(all_events, ignore_index=True)

# только убыточные события
loss_events = events[events["is_loss"]].copy()

# сводка
summary = (events.groupby("door")
          .agg(
              total_opens=("duration_min", "size"),
              loss_opens=("is_loss", "sum"),
              total_open_min=("duration_min", "sum"),
              total_loss_open_min=("duration_min", lambda s: s[events.loc[s.index, "is_loss"]].sum()),
              total_over_threshold_min=("over_threshold_min", "sum"),
              max_open_min=("duration_min", "max"),
          )
          .sort_values("loss_opens", ascending=False)
)

summary

,total_opens,loss_opens,total_open_min,total_loss_open_min,total_over_threshold_min,max_open_min
door,,,,,,
Дверь: Холодильник 1,1929,235,3007.183333,1937.983333,1232.983333,98.683333
Дверь: Холодильник 2,1073,152,1925.300000,1181.633333,725.633333,52.700000
Дверь: Морозильник 1,928,86,1390.050000,839.183333,581.183333,67.633333


In [6]:
RATE_KZT_PER_HOUR = 300  # пример

summary["loss_kzt"] = (summary["total_over_threshold_min"] / 60) * RATE_KZT_PER_HOUR
summary

,total_opens,loss_opens,total_open_min,total_loss_open_min,total_over_threshold_min,max_open_min,loss_kzt
door,,,,,,,
Дверь: Холодильник 1,1929,235,3007.183333,1937.983333,1232.983333,98.683333,6164.916667
Дверь: Холодильник 2,1073,152,1925.300000,1181.633333,725.633333,52.700000,3628.166667
Дверь: Морозильник 1,928,86,1390.050000,839.183333,581.183333,67.633333,2905.916667


In [7]:
EXTRA_POWER_KW = 0.8
TARIFF = 120

summary["loss_kzt"] = (summary["total_over_threshold_min"] / 60) * EXTRA_POWER_KW * TARIFF
summary

,total_opens,loss_opens,total_open_min,total_loss_open_min,total_over_threshold_min,max_open_min,loss_kzt
door,,,,,,,
Дверь: Холодильник 1,1929,235,3007.183333,1937.983333,1232.983333,98.683333,1972.773333
Дверь: Холодильник 2,1073,152,1925.300000,1181.633333,725.633333,52.700000,1161.013333
Дверь: Морозильник 1,928,86,1390.050000,839.183333,581.183333,67.633333,929.893333
